This notebook contains code relevant to converting categorical variables into one-hot encoded vectors and passing them as groups (all states belong to one variable is considered a group) into incremental r2 to calculate contribution.

Datasets we are using: 
1. Sleep/State with Intensity/Lifetime data
2. Running/Resting with Intensity/Lifetime data

# HELPER FUNCTIONS

In [3]:
# RFE linear regression (OLS) incremental r^2
# integer encoding of categorical predictors

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

def linear_RFE(X, y):
    rfe = RFE(LinearRegression(), importance_getter='auto')
    rfe = rfe.fit(X, y)
    rfe_ranking = [x for _, x in sorted(zip(rfe.ranking_, X.columns))]
    print('\nRFE Order:')
    print(rfe_ranking)
    return rfe_ranking

def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

def incremental_rSquared(X, y, featureRank, var_list, adjusted = True):
    print("Feature Rank: ", featureRank)
    print('\nIncremental R Squared:')
    increments = []
    rSquared = []
    rSquared_adjusted = []
    vars = []
    for feature in featureRank:
        print(feature)
        vars.append([i for i in var_list if i.startswith(feature)])
        vars = flatten_list(vars)

        print(vars)
        X_temp = X[vars].copy()
        model = LinearRegression()
        model.fit(X_temp, y)
        r_squared, adjusted_r_squared = calculate_rSquared(model, X_temp, y)
        
        rSquared.append(r_squared)
        rSquared_adjusted.append(adjusted_r_squared)
        
        if adjusted:
            increments.append(adjusted_r_squared-sum(increments))
        else:
            increments.append(r_squared-sum(increments))
    contribution = {featureRank[idx]:increments[idx]/rSquared[-1] for idx in range(len(increments))}
    return contribution, rSquared, rSquared_adjusted

def incremental_rSquared_categorical(X, y, featureRank, adjusted = True):
    print("Feature Rank: ", featureRank)
    print('\nIncremental R Squared:')
    increments = []
    rSquared = []
    rSquared_adjusted = []
    for feature in range(len(featureRank)):
        
        X_temp = X[featureRank[0:feature+1]].copy()
        model = smf.OLS(formula = "")
        # model.fit(X_temp, y)
        r_squared, adjusted_r_squared = calculate_rSquared(model, X_temp, y)
        
        rSquared.append(r_squared)
        rSquared_adjusted.append(adjusted_r_squared)
        
        if adjusted:
            increments.append(adjusted_r_squared-sum(increments))
        else:
            increments.append(r_squared-sum(increments))
    contribution = {featureRank[idx]:increments[idx]/rSquared[-1] for idx in range(len(increments))}
    return contribution, rSquared, rSquared_adjusted

def calculate_rSquared(model, X, y):
#     yhat = model.predict(X)
#     SS_Residual = sum((y-yhat)**2)       
#     SS_Total = sum((y-np.mean(y))**2)     
#     r_squared = 1 - (float(SS_Residual))/SS_Total
    r_squared = model.score(X,y)
    adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
    return r_squared, adjusted_r_squared


# SLEEP STATE DATASET

In [4]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# X is your input data matrix and y is your target variable
df_sleep = pd.read_csv('/content/sleep_wake_all_table.csv')
df_sleep_categorical = df_sleep.copy()

print('\n\nIntensity Data')

X = df_sleep_categorical.drop(['Intensity', 'Lifetime'], axis=1)
y = df_sleep_categorical['Intensity']

# Converting to one-hot encoded dataset
X_dummy_mouse = pd.get_dummies(X['Mouse_ID'], prefix='mouse')
X_dummy_time = pd.get_dummies(X['Time'], prefix='time')
X_dummy_state = pd.get_dummies(X['State'], prefix='state')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_time)
display(X_dummy_state)

X_sleep_dummy = pd.concat([X_dummy_mouse, X_dummy_time, X_dummy_state], axis = 1)
X_slee_dummy = X_sleep_dummy.drop(['mouse_1610', 'time_1', 'state_2'], axis = 1)
display(X_sleep_dummy)

X = X_sleep_dummy.copy()

# Assume groups is a list of lists, where each inner list represents a group of features
groups = [['mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630', 'mouse_1610'], ['time_2', 'time_3', 'time_1'], ['state_3', 'state_2']]
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=1)
ranking_intensity_sleep = []
vary = ['mouse', 'time', 'state']
count = 0
for group in groups:
    X_group = X[group]
    display(X_group)
    estimator.fit(X_group, y)
    r_squared, adjusted_r_squared = calculate_rSquared(estimator, X_group, y)
    print(r_squared)
    ranking_intensity_sleep.append(['{}'.format(vary[count]), r_squared])
    count = count + 1
    # Do something with the ranking, e.g., store it in a dictionary or print it


print("LIFETIME")
# Assume X is your input data matrix and y is your target variable
df_sleep = pd.read_csv('/content/sleep_wake_all_table.csv')
#df_sleep_categorical = integer_encoding(df_sleep, ['Mouse_ID', 'Time', 'State'])
df_sleep_categorical = df_sleep.copy()
X = df_sleep_categorical.drop(['Intensity', 'Lifetime'], axis=1)
y = df_sleep_categorical['Lifetime']

X_dummy_mouse = pd.get_dummies(X['Mouse_ID'], prefix='mouse')
X_dummy_time = pd.get_dummies(X['Time'], prefix='time')
X_dummy_state = pd.get_dummies(X['State'], prefix='state')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_time)
display(X_dummy_state)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_time, X_dummy_state], axis = 1)
# X_dummy = X_dummy.drop(['mouse_1610', 'time_1', 'state_2'], axis = 1)
display(X_dummy)

X = X_dummy.copy()

# Assume groups is a list of lists, where each inner list represents a group of features
groups = [['mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630', 'mouse_1610'], ['time_2', 'time_3', 'time_1'], ['state_3', 'state_2']]
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=1)
ranking_lieftime_sleep = []
vary = ['mouse', 'time', 'state']
count = 0
for group in groups:
    X_group = X[group]
    display(X_group)
    estimator.fit(X_group, y)
    r_squared, adjusted_r_squared = calculate_rSquared(estimator, X_group, y)
    print(r_squared)
    ranking_lieftime_sleep.append(['{}'.format(vary[count]), r_squared])
    count = count + 1
    # Do something with the ranking, e.g., store it in a dictionary or print it


print("Ranking Intensity: ", ranking_intensity_sleep)
print("Ranking Lifetime: ", ranking_lieftime_sleep)



Intensity Data
dataset prit


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
622,0,0,0,0,0,1
623,0,0,0,0,0,1
624,0,0,0,0,0,1
625,0,0,0,0,0,1


,time_1,time_2,time_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
622,0,0,1
623,0,0,1
624,0,0,1
625,0,0,1


,state_2,state_3
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
622,0,1
623,0,1
624,0,1
625,0,1


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630,time_1,time_2,time_3,state_2,state_3
0,1,0,0,0,0,0,0,1,0,1,0
1,1,0,0,0,0,0,0,1,0,1,0
2,1,0,0,0,0,0,0,1,0,1,0
3,1,0,0,0,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
622,0,0,0,0,0,1,0,0,1,0,1
623,0,0,0,0,0,1,0,0,1,0,1
624,0,0,0,0,0,1,0,0,1,0,1
625,0,0,0,0,0,1,0,0,1,0,1


...

0.26844132300778656


,time_2,time_3,time_1
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
622,0,1,0
623,0,1,0
624,0,1,0
625,0,1,0


0.09194011855262163


,state_3,state_2
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
622,1,0
623,1,0
624,1,0
625,1,0


0.4628627764398847
Ranking Intensity:  [['mouse', 0.6638799153737677], ['time', 0.2660355479543216], ['state', 0.0035282601163437732]]
Ranking Lifetime:  [['mouse', 0.26844132300778656], ['time', 0.09194011855262163], ['state', 0.4628627764398847]]


In [5]:
# Define the ranking based on the above values
intensity_rank = ['mouse', 'time', 'state']
lifetime_rank = ['state','mouse','time']

# Assume X is your input data matrix and y is your target variable
df_sleep = pd.read_csv('/content/sleep_wake_all_table.csv')
#df_sleep_categorical = integer_encoding(df_sleep, ['Mouse_ID', 'Time', 'State'])
df_sleep_categorical = df_sleep.copy()
print('\n\nIntensity Data')
X = df_sleep_categorical.drop(['Intensity', 'Lifetime'], axis=1)
y = df_sleep_categorical['Intensity']

X_dummy_mouse = pd.get_dummies(X['Mouse_ID'], prefix='mouse')
X_dummy_time = pd.get_dummies(X['Time'], prefix='time')
X_dummy_state = pd.get_dummies(X['State'], prefix='state')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_time)
display(X_dummy_state)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_time, X_dummy_state], axis = 1)
# X_dummy = X_dummy.drop(['mouse_1610', 'time_1', 'state_2'], axis = 1)
display(X_dummy)

X = X_dummy.copy()

var_list = X.columns.tolist()
contribution, r_squared, adjusted_r_squared = incremental_rSquared(X, y, intensity_rank, var_list)

print('\nR^2 Values:')
print(r_squared)
print('\nAdjusted R^2 Values:')
print(adjusted_r_squared)
print('\nContribution:')
print(contribution)




Intensity Data
dataset prit


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
622,0,0,0,0,0,1
623,0,0,0,0,0,1
624,0,0,0,0,0,1
625,0,0,0,0,0,1


,time_1,time_2,time_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
622,0,0,1
623,0,0,1
624,0,0,1
625,0,0,1


,state_2,state_3
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
622,0,1
623,0,1
624,0,1
625,0,1


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630,time_1,time_2,time_3,state_2,state_3
0,1,0,0,0,0,0,0,1,0,1,0
1,1,0,0,0,0,0,0,1,0,1,0
2,1,0,0,0,0,0,0,1,0,1,0
3,1,0,0,0,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
622,0,0,0,0,0,1,0,0,1,0,1
623,0,0,0,0,0,1,0,0,1,0,1
624,0,0,0,0,0,1,0,0,1,0,1
625,0,0,0,0,0,1,0,0,1,0,1


Feature Rank:  ['mouse', 'time', 'state']

Incremental R Squared:
mouse
['mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630']
time
['mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630', 'time_1', 'time_2', 'time_3']
state
['mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630', 'time_1', 'time_2', 'time_3', 'state_2', 'state_3']

R^2 Values:
[0.6641251692083051, 0.8197567683490964, 0.8301187090040819]

Adjusted R^2 Values:
[0.6608747676199984, 0.8171276126199908, 0.8270801818480573]

Contribution:
{'mouse': 0.7961207962808952, 'time': 0.18822951862806894, 'state': 0.011989332513667744}


In [6]:
intensity_rank = ['mouse', 'time', 'state']
lifetime_rank = ['state','mouse','time']

# Assume X is your input data matrix and y is your target variable
df_sleep = pd.read_csv('/content/sleep_wake_all_table.csv')
#df_sleep_categorical = integer_encoding(df_sleep, ['Mouse_ID', 'Time', 'State'])
df_sleep_categorical = df_sleep.copy()
print('\n\nIntensity Data')
X = df_sleep_categorical.drop(['Intensity', 'Lifetime'], axis=1)
y = df_sleep_categorical['Lifetime']

X_dummy_mouse = pd.get_dummies(X['Mouse_ID'], prefix='mouse')
X_dummy_time = pd.get_dummies(X['Time'], prefix='time')
X_dummy_state = pd.get_dummies(X['State'], prefix='state')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_time)
display(X_dummy_state)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_time, X_dummy_state], axis = 1)
# X_dummy = X_dummy.drop(['mouse_1610', 'time_1', 'state_2'], axis = 1)
display(X_dummy)

X = X_dummy.copy()

var_list = X.columns.tolist()
contribution, r_squared, adjusted_r_squared = incremental_rSquared(X, y, lifetime_rank, var_list)

print('\nR^2 Values:')
print(r_squared)
print('\nAdjusted R^2 Values:')
print(adjusted_r_squared)
print('\nContribution:')
print(contribution)




Intensity Data
dataset prit


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
622,0,0,0,0,0,1
623,0,0,0,0,0,1
624,0,0,0,0,0,1
625,0,0,0,0,0,1


,time_1,time_2,time_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
622,0,0,1
623,0,0,1
624,0,0,1
625,0,0,1


,state_2,state_3
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
622,0,1
623,0,1
624,0,1
625,0,1


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630,time_1,time_2,time_3,state_2,state_3
0,1,0,0,0,0,0,0,1,0,1,0
1,1,0,0,0,0,0,0,1,0,1,0
2,1,0,0,0,0,0,0,1,0,1,0
3,1,0,0,0,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
622,0,0,0,0,0,1,0,0,1,0,1
623,0,0,0,0,0,1,0,0,1,0,1
624,0,0,0,0,0,1,0,0,1,0,1
625,0,0,0,0,0,1,0,0,1,0,1


Feature Rank:  ['state', 'mouse', 'time']

Incremental R Squared:
state
['state_2', 'state_3']
mouse
['state_2', 'state_3', 'mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630']
time
['state_2', 'state_3', 'mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630', 'time_1', 'time_2', 'time_3']

R^2 Values:
[0.4628796322734756, 0.6985231950358295, 0.7760955847323004]

Adjusted R^2 Values:
[0.4611580926333265, 0.6946205826738338, 0.7720907903128782]

Contribution:
{'state': 0.5942027009371459, 'mouse': 0.3008166708241688, 'time': 0.09982044629949328}


# RVR DATASET

In [8]:
# Clubbing states for variables together

# RVR Data
# RFE linear regression (OLS) incremental r^2
# integer encoding of categorical predictors
# running = 1
# resting  = 0
df_categorical = pd.read_csv("/content/rvr.csv", converters={'Speed': lambda x: 1 if float(x)>0 else 0})
#df_categorical = integer_encoding(df_categorical, ['Mouse ID', 'Speed', 'Laser'])

print('\n\nIntensity Data')
X = df_categorical.drop(['Intensity', 'Fit Lifetime', 'Empirical Lifetime', 'Dataset'], axis=1)
y = df_categorical['Intensity']

X_dummy_mouse = pd.get_dummies(X['Mouse ID'], prefix='mouse')
X_dummy_speed = pd.get_dummies(X['Speed'], prefix='speed')
X_dummy_laser = pd.get_dummies(X['Laser'], prefix='laser')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_speed)
display(X_dummy_laser)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_speed, X_dummy_laser], axis = 1)
display(X_dummy)

X = X_dummy.copy()
print("Check")
display(X)

# Assume groups is a list of lists, where each inner list represents a group of features
groups = [['mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630'], ['laser_2', 'laser_3'], ['speed_1']]
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=1)
ranking_intensity_rvr = []
vary = ['mouse', 'laser', 'speed']
count = 0
for group in groups:
    X_group = X[group]
    display(X_group)
    estimator.fit(X_group, y)
    r_squared, adjusted_r_squared = calculate_rSquared(estimator, X_group, y)
    print(r_squared)
    ranking_intensity_rvr.append(['{}'.format(vary[count]), r_squared])
    count = count + 1

print("LIFETIME")
df_categorical = pd.read_csv("/content/rvr.csv", converters={'Speed': lambda x: 1 if float(x)>0 else 0})
#df_categorical = integer_encoding(df_categorical, ['Mouse ID', 'Speed', 'Laser'])

X = df_categorical.drop(['Intensity', 'Fit Lifetime', 'Empirical Lifetime', 'Dataset'], axis=1)
y = df_categorical['Empirical Lifetime']

X_dummy_mouse = pd.get_dummies(X['Mouse ID'], prefix='mouse')
X_dummy_speed = pd.get_dummies(X['Speed'], prefix='speed')
X_dummy_laser = pd.get_dummies(X['Laser'], prefix='laser')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_speed)
display(X_dummy_laser)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_speed, X_dummy_laser], axis = 1)
display(X_dummy)

X = X_dummy.copy()
print("Check")
display(X)

# Assume groups is a list of lists, where each inner list represents a group of features
groups = [['mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630', 'mouse_1610'], ['laser_1', 'laser_2', 'laser_3'], ['speed_1', 'speed_0']]
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=1)
ranking_lft_rvr = []
vary = ['mouse', 'laser', 'speed']
count = 0
for group in groups:
    X_group = X[group]
    display(X_group)
    estimator.fit(X_group, y)
    r_squared, adjusted_r_squared = calculate_rSquared(estimator, X_group, y)
    print(r_squared)
    ranking_lft_rvr.append(['{}'.format(vary[count]), r_squared])
    count = count + 1
    # Do something with the ranking, e.g., store it in a dictionary or print it

print("Ranking Intensity: ", ranking_intensity_rvr)
print("Ranking Lifetime: ", ranking_lft_rvr)



Intensity Data
dataset prit


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
584,0,0,1,0,0,0
585,0,0,1,0,0,0
586,0,0,1,0,0,0
587,0,0,1,0,0,0


,speed_0,speed_1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
584,0,1
585,0,1
586,0,1
587,0,1


,laser_1,laser_2,laser_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
584,0,1,0
585,0,1,0
586,0,1,0
587,0,1,0


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630,speed_0,speed_1,laser_1,laser_2,laser_3
0,0,1,0,0,0,0,1,0,0,1,0
1,0,1,0,0,0,0,1,0,0,1,0
2,0,1,0,0,0,0,1,0,0,1,0
3,0,1,0,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
584,0,0,1,0,0,0,0,1,0,1,0
585,0,0,1,0,0,0,0,1,0,1,0
586,0,0,1,0,0,0,0,1,0,1,0
587,0,0,1,0,0,0,0,1,0,1,0


...

0.23209886199071295


,laser_1,laser_2,laser_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
584,0,1,0
585,0,1,0
586,0,1,0
587,0,1,0


0.15852117325780946


,speed_1,speed_0
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
584,1,0
585,1,0
586,1,0
587,1,0


0.6544337778224301
Ranking Intensity:  [['mouse', 0.6410908088253346], ['laser', 0.21494782777619237], ['speed', 0.036861561107036755]]
Ranking Lifetime:  [['mouse', 0.23209886199071295], ['laser', 0.15852117325780946], ['speed', 0.6544337778224301]]


In [9]:
intensity_rank = ['mouse', 'laser', 'speed']
lifetime_rank = ['speed','laser','mouse']

df_categorical = pd.read_csv("/content/rvr.csv", converters={'Speed': lambda x: 1 if float(x)>0 else 0})
#df_categorical = integer_encoding(df_categorical, ['Mouse ID', 'Speed', 'Laser'])

print('\n\nIntensity Data')
X = df_categorical.drop(['Intensity', 'Fit Lifetime', 'Empirical Lifetime', 'Dataset'], axis=1)
y = df_categorical['Intensity']

X_dummy_mouse = pd.get_dummies(X['Mouse ID'], prefix='mouse')
X_dummy_speed = pd.get_dummies(X['Speed'], prefix='speed')
X_dummy_laser = pd.get_dummies(X['Laser'], prefix='laser')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_speed)
display(X_dummy_laser)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_speed, X_dummy_laser], axis = 1)
display(X_dummy)

X = X_dummy.copy()

var_list = X.columns.tolist()
contribution, r_squared, adjusted_r_squared = incremental_rSquared(X, y, intensity_rank, var_list)

print('\nR^2 Values:')
print(r_squared)
print('\nAdjusted R^2 Values:')
print(adjusted_r_squared)
print('\nContribution:')
print(contribution)




Intensity Data
dataset prit


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
584,0,0,1,0,0,0
585,0,0,1,0,0,0
586,0,0,1,0,0,0
587,0,0,1,0,0,0


,speed_0,speed_1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
584,0,1
585,0,1
586,0,1
587,0,1


,laser_1,laser_2,laser_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
584,0,1,0
585,0,1,0
586,0,1,0
587,0,1,0


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630,speed_0,speed_1,laser_1,laser_2,laser_3
0,0,1,0,0,0,0,1,0,0,1,0
1,0,1,0,0,0,0,1,0,0,1,0
2,0,1,0,0,0,0,1,0,0,1,0
3,0,1,0,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
584,0,0,1,0,0,0,0,1,0,1,0
585,0,0,1,0,0,0,0,1,0,1,0
586,0,0,1,0,0,0,0,1,0,1,0
587,0,0,1,0,0,0,0,1,0,1,0


Feature Rank:  ['mouse', 'laser', 'speed']

Incremental R Squared:
mouse
['mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630']
laser
['mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630', 'laser_1', 'laser_2', 'laser_3']
speed
['mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630', 'laser_1', 'laser_2', 'laser_3', 'speed_0', 'speed_1']

R^2 Values:
[0.6410337728185622, 0.8999379842164216, 0.9136419301627885]

Adjusted R^2 Values:
[0.6373330900641144, 0.8983826160954334, 0.9119955891433615]

Contribution:
{'mouse': 0.6975742564163593, 'laser': 0.28572410855181124, 'speed': 0.014899680715729294}


In [15]:
intensity_rank = ['mouse', 'laser', 'speed']
lifetime_rank = ['speed']

df_categorical = pd.read_csv("/content/rvr.csv", converters={'Speed': lambda x: 1 if float(x)>0 else 0})
#df_categorical = integer_encoding(df_categorical, ['Mouse ID', 'Speed', 'Laser'])

print('\n\Lifetime Data')
X = df_categorical.drop(['Intensity', 'Fit Lifetime', 'Empirical Lifetime', 'Dataset'], axis=1)
y = df_categorical['Empirical Lifetime']

X_dummy_mouse = pd.get_dummies(X['Mouse ID'], prefix='mouse')
X_dummy_speed = pd.get_dummies(X['Speed'], prefix='speed')
X_dummy_laser = pd.get_dummies(X['Laser'], prefix='laser')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_speed)
display(X_dummy_laser)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_speed, X_dummy_laser], axis = 1)
display(X_dummy)

X = X_dummy.copy()

var_list = X.columns.tolist()
contribution, r_squared, adjusted_r_squared = incremental_rSquared(X, y, lifetime_rank, var_list)

print('\nR^2 Values:')
print(r_squared)
print('\nAdjusted R^2 Values:')
print(adjusted_r_squared)
print('\nContribution:')
print(contribution)



\Lifetime Data
dataset prit


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
584,0,0,1,0,0,0
585,0,0,1,0,0,0
586,0,0,1,0,0,0
587,0,0,1,0,0,0


,speed_0,speed_1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
584,0,1
585,0,1
586,0,1
587,0,1


,laser_1,laser_2,laser_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
584,0,1,0
585,0,1,0
586,0,1,0
587,0,1,0


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630,speed_0,speed_1,laser_1,laser_2,laser_3
0,0,1,0,0,0,0,1,0,0,1,0
1,0,1,0,0,0,0,1,0,0,1,0
2,0,1,0,0,0,0,1,0,0,1,0
3,0,1,0,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
584,0,0,1,0,0,0,0,1,0,1,0
585,0,0,1,0,0,0,0,1,0,1,0
586,0,0,1,0,0,0,0,1,0,1,0
587,0,0,1,0,0,0,0,1,0,1,0


Feature Rank:  ['speed']

Incremental R Squared:
speed
['speed_0', 'speed_1']

R^2 Values:
[0.6544147647005483]

Adjusted R^2 Values:
[0.6532352929077174]

Contribution:
{'speed': 0.998197669342973}


In [14]:
intensity_rank = ['mouse', 'laser', 'speed']
lifetime_rank = ['speed','mouse']

df_categorical = pd.read_csv("/content/rvr.csv", converters={'Speed': lambda x: 1 if float(x)>0 else 0})
#df_categorical = integer_encoding(df_categorical, ['Mouse ID', 'Speed', 'Laser'])

print('\n\Lifetime Data')
X = df_categorical.drop(['Intensity', 'Fit Lifetime', 'Empirical Lifetime', 'Dataset'], axis=1)
y = df_categorical['Empirical Lifetime']

X_dummy_mouse = pd.get_dummies(X['Mouse ID'], prefix='mouse')
X_dummy_speed = pd.get_dummies(X['Speed'], prefix='speed')
X_dummy_laser = pd.get_dummies(X['Laser'], prefix='laser')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_speed)
display(X_dummy_laser)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_speed, X_dummy_laser], axis = 1)
display(X_dummy)

X = X_dummy.copy()

var_list = X.columns.tolist()
contribution, r_squared, adjusted_r_squared = incremental_rSquared(X, y, lifetime_rank, var_list)

print('\nR^2 Values:')
print(r_squared)
print('\nAdjusted R^2 Values:')
print(adjusted_r_squared)
print('\nContribution:')
print(contribution)



\Lifetime Data
dataset prit


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
584,0,0,1,0,0,0
585,0,0,1,0,0,0
586,0,0,1,0,0,0
587,0,0,1,0,0,0


,speed_0,speed_1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
584,0,1
585,0,1
586,0,1
587,0,1


,laser_1,laser_2,laser_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
584,0,1,0
585,0,1,0
586,0,1,0
587,0,1,0


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630,speed_0,speed_1,laser_1,laser_2,laser_3
0,0,1,0,0,0,0,1,0,0,1,0
1,0,1,0,0,0,0,1,0,0,1,0
2,0,1,0,0,0,0,1,0,0,1,0
3,0,1,0,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
584,0,0,1,0,0,0,0,1,0,1,0
585,0,0,1,0,0,0,0,1,0,1,0
586,0,0,1,0,0,0,0,1,0,1,0
587,0,0,1,0,0,0,0,1,0,1,0


Feature Rank:  ['speed', 'mouse']

Incremental R Squared:
speed
['speed_0', 'speed_1']
mouse
['speed_0', 'speed_1', 'mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630']

R^2 Values:
[0.6544147647005483, 0.7296321715531685]

Adjusted R^2 Values:
[0.6532352929077174, 0.7259029601263156]

Contribution:
{'speed': 0.8952939828806822, 'mouse': 0.09959493297000661}


In [16]:
intensity_rank = ['mouse', 'laser', 'speed']
lifetime_rank = ['speed','laser']

df_categorical = pd.read_csv("/content/rvr.csv", converters={'Speed': lambda x: 1 if float(x)>0 else 0})
#df_categorical = integer_encoding(df_categorical, ['Mouse ID', 'Speed', 'Laser'])

print('\n\Lifetime Data')
X = df_categorical.drop(['Intensity', 'Fit Lifetime', 'Empirical Lifetime', 'Dataset'], axis=1)
y = df_categorical['Empirical Lifetime']

X_dummy_mouse = pd.get_dummies(X['Mouse ID'], prefix='mouse')
X_dummy_speed = pd.get_dummies(X['Speed'], prefix='speed')
X_dummy_laser = pd.get_dummies(X['Laser'], prefix='laser')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_speed)
display(X_dummy_laser)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_speed, X_dummy_laser], axis = 1)
display(X_dummy)

X = X_dummy.copy()

var_list = X.columns.tolist()
contribution, r_squared, adjusted_r_squared = incremental_rSquared(X, y, lifetime_rank, var_list)

print('\nR^2 Values:')
print(r_squared)
print('\nAdjusted R^2 Values:')
print(adjusted_r_squared)
print('\nContribution:')
print(contribution)



\Lifetime Data
dataset prit


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
584,0,0,1,0,0,0
585,0,0,1,0,0,0
586,0,0,1,0,0,0
587,0,0,1,0,0,0


,speed_0,speed_1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
584,0,1
585,0,1
586,0,1
587,0,1


,laser_1,laser_2,laser_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
584,0,1,0
585,0,1,0
586,0,1,0
587,0,1,0


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630,speed_0,speed_1,laser_1,laser_2,laser_3
0,0,1,0,0,0,0,1,0,0,1,0
1,0,1,0,0,0,0,1,0,0,1,0
2,0,1,0,0,0,0,1,0,0,1,0
3,0,1,0,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
584,0,0,1,0,0,0,0,1,0,1,0
585,0,0,1,0,0,0,0,1,0,1,0
586,0,0,1,0,0,0,0,1,0,1,0
587,0,0,1,0,0,0,0,1,0,1,0


Feature Rank:  ['speed', 'laser']

Incremental R Squared:
speed
['speed_0', 'speed_1']
laser
['speed_0', 'speed_1', 'laser_1', 'laser_2', 'laser_3']

R^2 Values:
[0.6544147647005483, 0.8714215740222784]

Adjusted R^2 Values:
[0.6532352929077174, 0.8703188430962259]

Contribution:
{'speed': 0.7496202898587143, 'laser': 0.24911427104851386}


In [17]:
intensity_rank = ['mouse', 'laser', 'speed']
lifetime_rank = ['speed','laser','mouse']

df_categorical = pd.read_csv("/content/rvr.csv", converters={'Speed': lambda x: 1 if float(x)>0 else 0})
#df_categorical = integer_encoding(df_categorical, ['Mouse ID', 'Speed', 'Laser'])

print('\n\Lifetime Data')
X = df_categorical.drop(['Intensity', 'Fit Lifetime', 'Empirical Lifetime', 'Dataset'], axis=1)
y = df_categorical['Empirical Lifetime']

X_dummy_mouse = pd.get_dummies(X['Mouse ID'], prefix='mouse')
X_dummy_speed = pd.get_dummies(X['Speed'], prefix='speed')
X_dummy_laser = pd.get_dummies(X['Laser'], prefix='laser')

print("dataset prit")
display(X_dummy_mouse)
display(X_dummy_speed)
display(X_dummy_laser)

X_dummy = pd.concat([X_dummy_mouse, X_dummy_speed, X_dummy_laser], axis = 1)
display(X_dummy)

X = X_dummy.copy()

var_list = X.columns.tolist()
contribution, r_squared, adjusted_r_squared = incremental_rSquared(X, y, lifetime_rank, var_list)

print('\nR^2 Values:')
print(r_squared)
print('\nAdjusted R^2 Values:')
print(adjusted_r_squared)
print('\nContribution:')
print(contribution)




\Lifetime Data
dataset prit


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0
...,...,...,...,...,...,...
584,0,0,1,0,0,0
585,0,0,1,0,0,0
586,0,0,1,0,0,0
587,0,0,1,0,0,0


,speed_0,speed_1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
584,0,1
585,0,1
586,0,1
587,0,1


,laser_1,laser_2,laser_3
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
584,0,1,0
585,0,1,0
586,0,1,0
587,0,1,0


,mouse_1610,mouse_1611,mouse_1612,mouse_1626,mouse_1629,mouse_1630,speed_0,speed_1,laser_1,laser_2,laser_3
0,0,1,0,0,0,0,1,0,0,1,0
1,0,1,0,0,0,0,1,0,0,1,0
2,0,1,0,0,0,0,1,0,0,1,0
3,0,1,0,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
584,0,0,1,0,0,0,0,1,0,1,0
585,0,0,1,0,0,0,0,1,0,1,0
586,0,0,1,0,0,0,0,1,0,1,0
587,0,0,1,0,0,0,0,1,0,1,0


Feature Rank:  ['speed', 'laser', 'mouse']

Incremental R Squared:
speed
['speed_0', 'speed_1']
laser
['speed_0', 'speed_1', 'laser_1', 'laser_2', 'laser_3']
mouse
['speed_0', 'speed_1', 'laser_1', 'laser_2', 'laser_3', 'mouse_1610', 'mouse_1611', 'mouse_1612', 'mouse_1626', 'mouse_1629', 'mouse_1630']

R^2 Values:
[0.6544147647005483, 0.8714215740222784, 0.891280429538597]

Adjusted R^2 Values:
[0.6532352929077174, 0.8703188430962259, 0.8892077860809272]

Contribution:
{'speed': 0.732917801466692, 'laser': 0.24356368993863073, 'mouse': 0.021193041335463715}
